In [1]:
from Mollification import *

import trimesh
import os
import time

import meshio as mio

from buildGlueMap import *

root_folder = os.getcwd()

In [2]:
G = np.array([[[-1, -1], [-1, -1], [1, 0]], [[0,2], [2, 0], [3, 0]], [[1, 1], [-1, -1], [-1, -1]], [[1, 2], [-1, -1], [-1, -1]]])
FL = np.zeros((4, 3))
#E2FL = np.array([(0, 0), (0, 1), (0, 2), (1, 1), (2, 2), (2, 1), (1, 2), (3, 1), (3, 2)])
E2FL = np.array([[0, 0], [0, 1], [0, 2], [1, 1], [2, 2], [2, 1], [1, 2]])
#LE = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
LE = np.array([0, 1, 2, 3, 4, 5, 6])

FL2E = np.array([[[0], [1], [2]], [[2], [3], [6]], [[3], [5], [4]], [[6], [7], [8]]])



In [3]:
FL[E2FL[:,0], E2FL[:,1]] = LE
print(FL)

[[0. 1. 2.]
 [0. 3. 6.]
 [0. 5. 4.]
 [0. 0. 0.]]


In [4]:
((G[E2FL[:,0], E2FL[:,1]][:,0] != -1) & (G[E2FL[:,0], E2FL[:,1]][:,1] != -1))

array([False, False,  True,  True, False, False,  True])

In [5]:
FS = (E2FL[:,0], E2FL[:,1])
print(FS)

(array([0, 0, 0, 1, 2, 2, 1]), array([0, 1, 2, 1, 2, 1, 2]))


In [6]:
FSP = G[FS][((G[FS][:,0] != -1) & (G[FS][:,1] != -1)), :]
FSP = (FSP[:, 0], FSP[:, 1])
print(FSP)

(array([1, 2, 3]), array([0, 0, 0]))


In [7]:
FL[FSP] = LE[FL2E[FSP]][:,0]
print(FL)

[[0. 1. 2.]
 [2. 3. 6.]
 [3. 5. 4.]
 [6. 0. 0.]]


In [8]:
mesh = mio.read("../data/dime_counting_tube.stl")
F = mesh.cells_dict["triangle"]
V = mesh.points

In [9]:
delta = 1e-2
FL = igl.edge_lengths(V, F)

In [10]:
G = build_gluing_map(F)

end5 = time.time()

FLG, nMollG, nIterG = IntrinsicMollificationFL(FL, G, delta, scheme=MOLLIFICATION_SCHEME.GLOBAL_OPTIMIZATION_MANHATTAN)
end6 = time.time()
print("Global optimization with Manhattan distance took: ", end6 - end5, " seconds.")

FLL, nMollL, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.LOCAL_SCHEMES, local_scheme=MOLLIFICATION_LOCAL_SCHEME.ONE_BY_ONE_INTERPOLATED)

end7 = time.time()
print("Local optimization with one-by-one interpolation took: ", end7 - end6, " seconds.")

FLC, nMollC, _ = IntrinsicMollificationFL(FL, None, delta, scheme=MOLLIFICATION_SCHEME.CONSTANT_EPSILON)
end8 = time.time()
print("Constant epsilon took: ", end8 - end7, " seconds.")


Global optimization with Manhattan distance took:  0.00997018814086914  seconds.
Local optimization with one-by-one interpolation took:  0.0009982585906982422  seconds.
Constant epsilon took:  0.0009982585906982422  seconds.


In [11]:
np.linalg.norm(FLG - FL) / np.linalg.norm(FL)

0.8320499163879893

In [12]:
np.linalg.norm(FLL - FL)/ np.linalg.norm(FL)

0.0016614129683786927

In [13]:
np.linalg.norm(FLC - FL) / np.linalg.norm(FL)

0.005851617744390041